In [8]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import davies_bouldin_score, silhouette_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

Berikut adalah analisis singkat tentang setiap library yang digunakan:

1. pandas (import pandas as pd):
   - Pandas adalah library Python yang digunakan untuk manipulasi dan analisis data.
   - Penggunaannya untuk membuat dan mengelola DataFrame yang berisi data dari dataset 20newsgroups.
2. numpy (import numpy as np):
   - NumPy adalah library fundamental dalam ekosistem Python untuk komputasi numerik.
   - Penggunaannya untuk operasi numerik, termasuk array dan matriks numerik.
3. scipy.sparse (from scipy.sparse import csr_matrix):
   - SciPy adalah library yang menyediakan berbagai alat dan fungsi untuk komputasi ilmiah.
   - Penggunaan `csr_matrix` dari scipy.sparse untuk mengubah array fitur menjadi matriks sparse dalam format CSR (Compressed Sparse Row).
4. sklearn.feature_extraction.text (from sklearn.feature_extraction.text import CountVectorizer):
   - Bagian dari scikit-learn yang berfokus pada ekstraksi fitur dari data teks.
   - Penggunaan `CountVectorizer` untuk mengubah teks dokumen menjadi vektor fitur berdasarkan frekuensi kata.
5. sklearn.cluster (from sklearn.cluster import KMeans):
   - Modul scikit-learn untuk clustering data.
   - Penggunaan `KMeans` untuk melakukan K-Means Clustering.
6. sklearn.datasets (from sklearn.datasets import fetch_20newsgroups):
   - Modul scikit-learn yang menyediakan akses ke dataset yang umum digunakan dalam machine learning.
   - Penggunaan `fetch_20newsgroups` untuk memuat dataset 20newsgroups.
7. sklearn.metrics (from sklearn.metrics import davies_bouldin_score, silhouette_score):
   - Modul scikit-learn yang berisi berbagai metrik evaluasi untuk model machine learning dan clustering.
   - Penggunaan `davies_bouldin_score` dan `silhouette_score` untuk mengukur kualitas clustering.
8. sklearn.metrics.pairwise (from sklearn.metrics.pairwise import cosine_similarity):
   - Modul scikit-learn yang menyediakan metrik jarak (similarity) untuk data.
   - Penggunaan `cosine_similarity` untuk menghitung Cosine Similarity antara dokumen dalam dataset.
9. sklearn.metrics (from sklearn.metrics import pairwise_distances):
   - Modul scikit-learn yang digunakan untuk menghitung jarak antara dua matriks.
   - Penggunaan `pairwise_distances` untuk menghitung jarak antara dokumen berdasarkan Cosine Similarity.


In [9]:
# Definisikan kategori Sport, Religion, dan Technology
categories = [
    'comp.graphics',
    'rec.sport.baseball',
    'soc.religion.christian',
]

# Ambil dataset dari 20newsgroups
dataset = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, remove=('headers', 'footers', 'quotes'))
df = pd.DataFrame(dataset.data, columns=["corpus"])
df

,corpus
0,"\n\n\n\nAs far as I can see, one of the big di..."
1,My family has never been particularly religiou...
2,Which Version of the Bible do you consider to ...
3,We need a good concave ->convex polygon conver...
4,"Hello,\n I've recently had Povray draw abou..."
...,...
1775,"\n\n\nVery interesting, but I also believe tha..."
1776,Does anybody know what Delaunay Triangulation ...
1777,Here is some material by Michael Davies on the...
1778,"\nThat would be neat, but nowhere in the Bible..."


In [10]:
# Menggunakan Count Vectorizer untuk mengubah teks dokumen menjadi vektor fitur
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['corpus'])

# Ubah array X menjadi matriks sparse CSR
X_sparse = csr_matrix(X)

# Normalisasi matriks sparse 
from sklearn.preprocessing import normalize
X_normalized_sparse = normalize(X_sparse, norm='l2')

# Melakukan K-Means Clustering dengan Euclidean Distance
n_clusters = 3  # Ganti dengan jumlah cluster yang sesuai
kmeans_euclidean = KMeans(n_clusters=n_clusters, random_state=0).fit(X_normalized_sparse)
labels_euclidean = kmeans_euclidean.labels_

print('Hasil K-Means Clustering dengan Euclidean Distance :', labels_euclidean)

C:\Users\bayuk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Hasil K-Means Clustering dengan Euclidean Distance : [1 1 2 ... 2 1 2]


K-Means Clustering dengan metode Euclidean Distance pada data yang telah dinormalisasi

1. Count Vectorizer: Menggunakan `CountVectorizer` untuk mengubah teks dokumen menjadi vektor fitur berdasarkan frekuensi kata-kata. Ini mengubah teks dokumen menjadi representasi numerik yang dapat digunakan untuk clustering.
2. Matriks Sparse (X_sparse): Hasil dari `CountVectorizer` adalah matriks sparse yang menggambarkan frekuensi kata dalam dokumen. Matriks ini digunakan untuk meneruskan informasi ke algoritma clustering.
3. Normalisasi (X_normalized_sparse): Melakukan normalisasi pada matriks sparse berdasarkan Cosine Similarity. Normalisasi ini penting ketika ingin menggunakan metrik jarak Cosine Similarity dalam clustering.
4. K-Means Clustering dengan Euclidean Distance: Melakukan K-Means Clustering dengan Euclidean Distance menggunakan `KMeans`. Dalam menentukan jumlah cluster (n_clusters) yang ingin dihasilkan, dalam hal ini 3 cluster. Hasil clustering disimpan dalam `labels_euclidean`.


In [11]:
# Menghitung Cosine Similarity
cosine_similarities = cosine_similarity(X_normalized_sparse)

# Mengisi diagonal dengan nol pada matriks jarak Cosine Similarity
np.fill_diagonal(cosine_similarities, 0)

# Melakukan K-Means Clustering dengan Cosine Similarity
kmeans_cosine = KMeans(n_clusters=n_clusters, init='k-means++').fit(cosine_similarities)
labels_cosine = kmeans_cosine.labels_

print('Hasil K-Means Clustering dengan Cosine Similarity :', labels_cosine)

C:\Users\bayuk\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Hasil K-Means Clustering dengan Cosine Similarity : [0 0 1 ... 2 2 1]


K-Means Clustering dengan metode Cosine Similarity pada data yang telah dinormalisasi

1. Menghitung Cosine Similarity: Menggunakan `cosine_similarity` untuk menghitung Cosine Similarity antara vektor-vektor yang telah dinormalisasi dalam matriks `X_normalized_sparse`. Ini menghasilkan matriks jarak Cosine Similarity antara semua pasangan dokumen dalam dataset.
2. Mengisi Diagonal dengan Nol: Mengisi diagonal matriks jarak Cosine Similarity dengan nol menggunakan `np.fill_diagonal(cosine_similarities, 0)`. Hal ini dilakukan karena diagonal matriks tersebut akan selalu memiliki nilai 1 (similarity maksimum antara dokumen dengan diri mereka sendiri), dan dalam analisis clustering, biasanya diagonal diabaikan.
3. K-Means Clustering dengan Cosine Similarity: Melakukan K-Means Clustering dengan metode Cosine Similarity menggunakan `KMeans`. Menggunakan `cosine_similarities` sebagai data input. Jumlah cluster yang ditentukan adalah `n_clusters` (dalam hal ini, 3 cluster). Hasil clustering disimpan dalam `labels_cosine`.


In [12]:
# Mengelompokkan dokumen ke dalam 3 kelas (atau sesuai dengan n_clusters yang Anda tentukan)
for i in range(n_clusters):
    print(f"Kelas {i + 1} (Euclidean Distance):")
    for j in np.where(labels_euclidean == i)[0][:5]:
        print(f"- Dokumen {j + 1}: {df['corpus'][j]}")

for i in range(n_clusters):
    print(f"Kelas {i + 1} (Cosine Similarity):")
    for j in np.where(labels_cosine == i)[0][:5]:
        print(f"- Dokumen {j + 1}: {df['corpus'][j]}")

Kelas 1 (Euclidean Distance):
- Dokumen 9: I was watching the Dodgers/Marlins game yesterday and a couple of
things impressed me.  

First is that the way the sun was shining in Miami, it had a summer
atmosphere in early spring for baseball.  In comparison Wrigley Field
in early April still has a wintry look to it with the dead ivy and
bundled up fans.  

The second and most important was the fans.  I like these guys/gals!
I will admit I am a football fan first but I still enjoy baseball.
It was interesting because most of these fans are only accustomed to
the Miami Dolphins.  The way they were cheering, I thought it was the
AFC playoffs.  Of course opening day may have A LOT to do with it,
but I really got feeling of electricity that I think is lacking with
a lot of baseball fans in other cities.  

Baseball certainly needs a charge and I hope these two expansion
teams bring back some excitement.  We'll find out Friday how Denver
Bronco fans respond. 
- Dokumen 10: 

If you think that

Mengelompokkan dokumen ke dalam kelas (cluster) berdasarkan hasil clustering yang telah dilakukan baik dengan metode Euclidean Distance maupun Cosine Similarity.
1. For Loop untuk Kelas (Clusters): Loop `for` digunakan untuk iterasi sebanyak `n_clusters` kali. `n_clusters` adalah jumlah cluster yang telah ditentukan sebelumnya (dalam contoh ini, 3 cluster).
2. Mencetak Kelas (Clusters): Pada setiap iterasi, kode mencetak informasi tentang kelas (cluster) yang sedang diproses, baik untuk metode Euclidean Distance maupun Cosine Similarity.
3. For Loop untuk Dokumen: Setelah mencetak informasi tentang kelas, ada loop `for` lagi untuk iterasi sebanyak 5 kali. Ini bertujuan untuk mencetak informasi tentang 5 dokumen pertama yang termasuk dalam kelas tersebut.

Hasilnya: Untuk setiap kelas, kode mencetak informasi tentang kelas tersebut (misalnya, "Kelas 1 (Euclidean Distance)") dan kemudian mencetak 5 dokumen pertama yang termasuk dalam kelas tersebut. Ini membantu melihat bagaimana dokumen-dokumen dikelompokkan dalam setiap cluster.

In [13]:
# Menghitung DBI dan Silhouette Score
dbi_euclidean = davies_bouldin_score(X_normalized_sparse.toarray(), labels_euclidean)
dbi_cosine = davies_bouldin_score(cosine_similarities, labels_cosine)
silhouette_euclidean = silhouette_score(X_normalized_sparse.toarray(), labels_euclidean, metric='euclidean')
silhouette_cosine = silhouette_score(pairwise_distances(cosine_similarities, metric='cosine'), labels_cosine, metric='precomputed')

# Menampilkan hasil analisis
print("DBI (Euclidean Distance):", dbi_euclidean)
print("DBI (Cosine Similarity):", dbi_cosine)
print("Silhouette Score (Euclidean Distance):", silhouette_euclidean)
print("Silhouette Score (Cosine Similarity):", silhouette_cosine)

# Analisis hasil berdasarkan DBI dan Silhouette Score
if dbi_euclidean < dbi_cosine:
    print("Metode Euclidean Distance lebih baik berdasarkan DBI.")
else:
    print("Metode Cosine Similarity lebih baik berdasarkan DBI.")

if silhouette_euclidean > silhouette_cosine:
    print("Metode Euclidean Distance lebih baik berdasarkan Silhouette Score.")
else:
    print("Metode Cosine Similarity lebih baik berdasarkan Silhouette Score.")

DBI (Euclidean Distance): 8.15170299550008
DBI (Cosine Similarity): 2.4772374297512485
Silhouette Score (Euclidean Distance): 0.014445223697533108
Silhouette Score (Cosine Similarity): -0.020892957021890875
Metode Cosine Similarity lebih baik berdasarkan DBI.
Metode Euclidean Distance lebih baik berdasarkan Silhouette Score.


Melakukan evaluasi clustering berdasarkan dua metrik, yaitu Davies-Bouldin Index (DBI) dan Silhouette Score. Serta membandingkan metode Euclidean Distance dengan Cosine Similarity.

1. Menghitung DBI (Davies-Bouldin Index):
   - `dbi_euclidean` menghitung DBI untuk metode Euclidean Distance dengan memasukkan matriks data yang telah dinormalisasi (`X_normalized_sparse.toarray()`) dan label-label clustering (`labels_euclidean`).
   - `dbi_cosine` menghitung DBI untuk metode Cosine Similarity dengan memasukkan matriks jarak Cosine Similarity (`cosine_similarities`) dan label-label clustering (`labels_cosine`).
2. Menghitung Silhouette Score:
   - `silhouette_euclidean` menghitung Silhouette Score untuk metode Euclidean Distance dengan memasukkan matriks data yang telah dinormalisasi (`X_normalized_sparse.toarray()`), label-label clustering (`labels_euclidean`), dan metric 'euclidean'.
   - `silhouette_cosine` menghitung Silhouette Score untuk metode Cosine Similarity dengan memasukkan matriks jarak Cosine Similarity (`cosine_similarities`), label-label clustering (`labels_cosine`), dan metric 'precomputed'.
3. Menampilkan Hasil Analisis:
   - Mencetak hasil DBI dan Silhouette Score untuk kedua metode clustering (Euclidean Distance dan Cosine Similarity).
4. Analisis Berdasarkan DBI dan Silhouette Score:
   - Membandingkan hasil DBI antara metode Euclidean Distance dan Cosine Similarity. Jika DBI Euclidean Distance lebih rendah, maka ia dianggap lebih baik dalam hal kualitas clustering berdasarkan metrik ini.
   - Membandingkan hasil Silhouette Score antara kedua metode. Jika Silhouette Score Euclidean Distance lebih tinggi, maka metode ini dianggap lebih baik dalam hal kualitas clustering berdasarkan metrik ini.

Hasil evaluasi clustering berdasarkan Davies-Bouldin Index (DBI) dan Silhouette Score untuk metode Euclidean Distance dan Cosine Similarity:

1. DBI (Davies-Bouldin Index):
   - DBI (Euclidean Distance): 8.15170299550008
   - DBI (Cosine Similarity): 2.4772374297512485
   - Kesimpulan: Nilai DBI adalah sebuah angka yang mengindikasikan seberapa baik kualitas clustering. Semakin rendah nilai DBI, semakin baik kualitas clusteringnya. Dalam hal ini, nilai DBI untuk metode Cosine Similarity (2.4772) lebih rendah daripada Euclidean Distance (8.1517), menunjukkan bahwa metode Cosine Similarity menghasilkan clustering yang lebih baik dari pada Euclidean Distance berdasarkan metrik ini.

2. Silhouette Score:
   - Silhouette Score (Euclidean Distance): 0.014445223697533108
   - Silhouette Score (Cosine Similarity): -0.020892957021890875
   - Kesimpulan: Silhouette Score mengukur seberapa baik objek-data ditempatkan dalam cluster. Nilai Silhouette Score berkisar antara -1 hingga 1. Nilai positif menunjukkan bahwa objek-data ditempatkan dengan baik dalam cluster mereka, sedangkan nilai negatif menunjukkan bahwa objek-data mungkin ditempatkan dengan lebih baik dalam cluster lain. Dalam hal ini, metode Euclidean Distance memiliki Silhouette Score yang positif (0.0144), sedangkan metode Cosine Similarity memiliki Silhouette Score yang lebih rendah dan negatif (-0.0209). Ini mengindikasikan bahwa metode Euclidean Distance mungkin lebih baik dalam menempatkan objek-data dalam cluster mereka.

Kesimpulan:
Berdasarkan evaluasi DBI, metode Cosine Similarity lebih baik dalam hal kualitas clustering. Namun, berdasarkan evaluasi Silhouette Score, metode Euclidean Distance tampaknya lebih baik dalam menempatkan objek-data dalam cluster mereka. Oleh karena itu, kesimpulan dari penelitian ini dapat bervariasi tergantung pada metrik evaluasi yang lebih diutamakan. Dalam pengambilan keputusan akhir, penting untuk mempertimbangkan tujuan analisis clustering dan karakteristik data yang sedang diproses. Jika tujuan adalah untuk memaksimalkan kualitas clustering berdasarkan DBI, maka metode Cosine Similarity mungkin lebih disarankan. Namun, jika prioritas adalah memastikan objek-data ditempatkan dengan baik dalam cluster mereka, maka metode Euclidean Distance mungkin lebih sesuai. 